In [ ]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [ ]:
with sqlite3.connect("../db.sqlite3") as connection:
    cursor = connection.cursor()

    cursor.execute("""
        SELECT product.name AS name, 
               product.price AS price, 
               orderitem.quantity AS quantity, 
               "order".datetime 
        FROM restaurant_orderitem AS orderitem
        INNER JOIN restaurant_product AS product 
        ON orderitem.product_id = product.id
        
        INNER JOIN restaurant_order AS "order" 
        ON orderitem.order_id = "order".id
    """)
    rows = cursor.fetchall()
df = pd.DataFrame(rows, columns=["name", "price", "quantity", "datetime"])
df.to_csv("restaurant_orders.csv", index=False)

# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
with sqlite3.connect("../db.sqlite3") as connection:
    query = """
        SELECT product.name AS name, 
               SUM(orderitem.quantity) AS total_quantity
        FROM restaurant_orderitem AS orderitem
        INNER JOIN restaurant_product AS product 
        ON orderitem.product_id = product.id
        GROUP BY product.name
        ORDER BY total_quantity DESC
        LIMIT 10
    """
    top_products = pd.read_sql_query(query, connection)

plt.figure(figsize=(8, 8))
plt.pie(
    top_products["total_quantity"],
    labels=top_products["name"],
    autopct="%1.1f%%"
)
plt.title("Top 10 Most Popular Products by Quantity Sold")
plt.show()

# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
with sqlite3.connect("../db.sqlite3") as connection:
    query = """
        SELECT product.name AS name, 
               product.price * orderitem.quantity AS item_price
        FROM restaurant_orderitem AS orderitem
        INNER JOIN restaurant_product AS product 
        ON orderitem.product_id = product.id
    """
    df_orders = pd.read_sql_query(query, connection)

top_revenue_products = (
    df_orders.groupby("name")["item_price"]
    .sum()
    .nlargest(10)
)

top_revenue_products.plot.pie(
    autopct="%1.1f%%",
    title="Top 10 Products by Revenue"
)

# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:

with sqlite3.connect("../db.sqlite3") as connection:
    query = """
        SELECT product.name AS name, 
               product.price * orderitem.quantity AS item_price, 
               "order".datetime AS order_datetime
        FROM restaurant_orderitem AS orderitem
        INNER JOIN restaurant_product AS product ON orderitem.product_id = product.id
        INNER JOIN restaurant_order AS "order" ON orderitem.order_id = "order".id
    """
    df_orders = pd.read_sql_query(query, connection)
df_orders["order_datetime"] = pd.to_datetime(df_orders["order_datetime"])

df_orders["order_hour"] = df_orders["order_datetime"].dt.hour

income_by_hour = df_orders.groupby("order_hour")["item_price"].sum()

plt.figure(figsize=(10, 6))
income_by_hour.plot(kind="bar", color="green")
plt.title("Total Restaurant Income by Hour of Order")
plt.xlabel("Hour of Order (0-23)")
plt.ylabel("Total Income ($)")
plt.xticks(rotation=0)
plt.tight_layout()

plt.show()


# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
df_orders = pd.read_sql_query(query, connection)
df_orders["order_datetime"] = pd.to_datetime(df_orders["order_datetime"])
df_orders["order_day_of_week"] = df_orders["order_datetime"].dt.weekday
income_by_day = df_orders.groupby("order_day_of_week")["item_price"].sum()
days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]

plt.figure(figsize=(10, 6))
income_by_day.plot(kind="bar", color="lightcoral")
plt.title("Total Restaurant Income by Day of the Week")
plt.xlabel("Day of the Week")
plt.ylabel("Total Income ($)")
plt.xticks(ticks=range(7), labels=days_of_week, rotation=45)
plt.tight_layout()
plt.show()